# Demonstration of anomaly detection with CVAE using DASHlink data

**Author: Milad Memarzadeh (milad.memarzadeh@nasa.gov)**


# Import Libraries

In [81]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_curve, precision_recall_curve, average_precision_score, f1_score, auc
from sklearn.utils import shuffle
import warnings
import random

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


# Load and preprocess data

In [82]:
# Define the path to the dataset
#data_dir = '/home/adlink3/Downloads/yahoo_s5/A2Benchmark/'
data_dir = 'C:/Users/jed95/Documents/GitHub/anomaly_detection/dataset/yahoo_s5/A2Benchmark'
# Get the list of all CSV files in the directory
file_list = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]

# Load all files into a single DataFrame
df_list = []
for file in file_list:
    df = pd.read_csv(file)
    df_list.append(df)

data = pd.concat(df_list, ignore_index=True)
print("Data shape:", data.shape)


Data shape: (142100, 3)


In [83]:
# Check for missing values
print("Missing values:", data.isnull().sum())

# For simplicity, drop missing values (if any)
data.dropna(inplace=True)


Missing values: timestamp     0
value         0
is_anomaly    0
dtype: int64


In [84]:
scaler = MinMaxScaler()
data['value'] = scaler.fit_transform(data['value'].values.reshape(-1, 1))


# Create Sequences

In [85]:
def create_sequences(values, labels, sequence_length):
    sequences = []
    seq_labels = []
    for i in range(len(values) - sequence_length + 1):
        seq = values[i:i + sequence_length]
        label = labels[i + sequence_length - 1]  # Label of the last element in the sequence
        sequences.append(seq)
        seq_labels.append(label)
    return np.array(sequences), np.array(seq_labels)

sequence_length = 10  # You can adjust this 
values = data['value'].values
labels = data['is_anomaly'].values  # Assuming 'is_anomaly' is the label column

sequences, seq_labels = create_sequences(values, labels, sequence_length)
print("Sequences shape:", sequences.shape)
print("Sequence labels shape:", seq_labels.shape)



Sequences shape: (142091, 10)
Sequence labels shape: (142091,)


# Split data into labeled and unlabeled sets

In [86]:
# Split data into labeled and unlabeled sets
labeled_fraction = 0.5  # 10% labeled data
num_labeled = int(len(sequences) * labeled_fraction)

# Shuffle data before splitting
indices = np.arange(len(sequences))
np.random.shuffle(indices)
sequences = sequences[indices]
labels = labels[indices]

labeled_data = sequences[:num_labeled]
labeled_labels = labels[:num_labeled]

unlabeled_data = sequences[num_labeled:]
unlabeled_labels = labels[num_labeled:]  # For evaluation purposes


In [87]:
# Get indices for normal and anomalous sequences
#normal_indices = np.where(seq_labels == 0)[0]
#anomalous_indices = np.where(seq_labels == 1)[0]
#
## Extract normal and anomalous sequences
#normal_sequences = sequences[normal_indices]
#normal_labels = seq_labels[normal_indices]
#
#anomalous_sequences = sequences[anomalous_indices]
#anomalous_labels = seq_labels[anomalous_indices]


In [88]:
# Split normal data
#X_train_normal, X_test_normal, y_train_normal, y_test_normal = train_test_split(
#    normal_sequences, normal_labels, test_size=0.51, random_state=42, stratify=normal_labels)
#
## Split anomalous data
#X_train_anomalous, X_test_anomalous, y_train_anomalous, y_test_anomalous = train_test_split(
#    anomalous_sequences, anomalous_labels, test_size=0.51, random_state=42, stratify=anomalous_labels)


In [89]:
## Combine training data
#X_train = np.concatenate([X_train_normal, X_train_anomalous], axis=0)
#y_train = np.concatenate([y_train_normal, y_train_anomalous], axis=0)
#
## Combine test data
#X_test = np.concatenate([X_test_normal, X_test_anomalous], axis=0)
#y_test = np.concatenate([y_test_normal, y_test_anomalous], axis=0)
#print("Training data shape:", X_train.shape, y_train.shape)
#
#print("Test data shape:", X_test.shape, y_test.shape)
#

In [90]:
## Shuffle data
#
#X_train, y_train = shuffle(X_train, y_train, random_state=42)
#X_test, y_test = shuffle(X_test, y_test, random_state=42)
#

# Convert Data to Tensors

In [91]:
# Convert training data to tensors
#X_train_tensor = torch.tensor(X_train).float()
#y_train_tensor = torch.tensor(y_train).long()
#
## Convert test data to tensors
#X_test_tensor = torch.tensor(X_test).float()
#y_test_tensor = torch.tensor(y_test).long()


# Create Data Loaders for Labeled and Unlabeled Data

In [92]:
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences, labels=None):
        self.sequences = sequences
        self.labels = labels
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = torch.FloatTensor(self.sequences[idx])
        if self.labels is not None:
            label = torch.LongTensor([self.labels[idx]])
            return seq, label
        else:
            return seq


In [93]:
#batch_size = 64  # Adjust as needed
#
## Training data loader
#train_dataset = TimeSeriesDataset(X_train_tensor, y_train_tensor)
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False) #TODO adjust Shuffle training data? 
#
## Test data loader
#test_dataset = TimeSeriesDataset(X_test_tensor, y_test_tensor)
#test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Instantiate and Train the Conditional VAE

In [94]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim, num_classes=2):
        super(VAE, self).__init__()
        self.num_classes = num_classes
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.2)
        )
        self.fc_mu = nn.Linear(64, latent_dim)
        self.fc_logvar = nn.Linear(64, latent_dim)
        self.classifier = nn.Linear(64, num_classes)
        
        # Decoder
        self.decoder_input = nn.Linear(latent_dim + num_classes, 64)
        self.decoder = nn.Sequential(
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )
        
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
        
    def forward(self, x, y_onehot=None):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        
        class_logits = self.classifier(h)
        if y_onehot is None:
            y_onehot = F.softmax(class_logits, dim=1)
            
        z_cond = torch.cat([z, y_onehot], dim=1)
        x_recon = self.decoder_input(z_cond)
        x_recon = self.decoder(x_recon)
        return x_recon, mu, logvar, class_logits


In [95]:
def loss_function(x_recon, x, mu, logvar, class_logits, labels):
    recon_loss = F.mse_loss(x_recon, x, reduction='mean')
    kld = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    class_loss = F.cross_entropy(class_logits, labels.view(-1))
    return recon_loss + kld + class_loss, recon_loss, kld, class_loss

def train_epoch(model, dataloader, optimizer):
    model.train()
    total_loss = 0
    total_recon_loss = 0
    total_kld = 0
    total_class_loss = 0
    for batch in dataloader:
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        optimizer.zero_grad()
        x_recon, mu, logvar, class_logits = model(x_batch)
        loss, recon_loss, kld, class_loss = loss_function(x_recon, x_batch, mu, logvar, class_logits, y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_recon_loss += recon_loss.item()
        total_kld += kld.item()
        total_class_loss += class_loss.item()
    avg_loss = total_loss / len(dataloader)
    avg_recon_loss = total_recon_loss / len(dataloader)
    avg_kld = total_kld / len(dataloader)
    avg_class_loss = total_class_loss / len(dataloader)
    return avg_loss, avg_recon_loss, avg_kld, avg_class_loss


In [96]:
# Set up k-fold cross-validation
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# Lists to store metrics
fold_train_losses = []
fold_val_losses = []
fold_val_f1_scores = []

# Convert labeled data to dataset
full_dataset = TimeSeriesDataset(labeled_data, labeled_labels)

# Define input dimensions
input_dim = sequence_length # Since each sequence has length equal to window_size
latent_dim = 20  # Adjust as needed

# Perform k-fold cross-validation
for fold, (train_idx, val_idx) in enumerate(skf.split(labeled_data, labeled_labels)):
    print(f'\nFold {fold+1}/{k_folds}')
    print('--------------------------------')
    
    # Split data
    train_data = labeled_data[train_idx]
    train_labels = labeled_labels[train_idx]
    val_data = labeled_data[val_idx]
    val_labels = labeled_labels[val_idx]
    
    # Create datasets and dataloaders
    train_dataset = TimeSeriesDataset(train_data, train_labels)
    val_dataset = TimeSeriesDataset(val_data, val_labels)
    
    batch_size = 64  # Adjust as needed
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize model and optimizer
    model = VAE(input_dim, latent_dim).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    num_epochs = 10  # Adjust as needed
    
    best_val_f1 = 0
    epochs_no_improve = 0
    patience = 3  # Early stopping patience
    
    for epoch in range(num_epochs):
        # Training
        train_loss, train_recon_loss, train_kld, train_class_loss = train_epoch(model, train_loader, optimizer)
        
        # Validation
        model.eval()
        val_loss = 0
        val_true = []
        val_pred = []
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                x_recon, mu, logvar, class_logits = model(x_batch)
                loss, recon_loss, kld, class_loss = loss_function(x_recon, x_batch, mu, logvar, class_logits, y_batch)
                val_loss += loss.item()
                
                predictions = class_logits.argmax(dim=1)
                val_true.extend(y_batch.cpu().numpy())
                val_pred.extend(predictions.cpu().numpy())
        avg_val_loss = val_loss / len(val_loader)
        val_f1 = f1_score(val_true, val_pred, average='macro')
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val F1-Score: {val_f1:.4f}')
        
        # Early stopping
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            epochs_no_improve = 0
            # Save the best model for this fold
            torch.save(model.state_dict(), f'best_model_fold_{fold+1}.pth')
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print('Early stopping!')
                break
    
    # Load the best model for this fold
    model.load_state_dict(torch.load(f'best_model_fold_{fold+1}.pth'))
    
    # Store metrics for this fold
    fold_train_losses.append(train_loss)
    fold_val_losses.append(avg_val_loss)
    fold_val_f1_scores.append(best_val_f1)



Fold 1/5
--------------------------------
Epoch 1, Train Loss: 0.0505, Val Loss: 0.0228, Val F1-Score: 0.4993


KeyboardInterrupt: 

In [ ]:
from source.modelsCondVAE import *
from source.utilsCondVAEs5 import *

# Instantiate the model
latent_dim = 10  # Adjust as needed
num_param = 10    # Since we have univariate time series
window_size = 1
num_classes = 2  # Normal and Anomaly
scale_flag = 0   # Use Sigmoid activation in the decoder

model = VAE(latent_dim, num_param, window_size, num_classes, scale_flag).to(device)

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Train the model
num_epochs = 10  # Adjust as needed
train_model_full(model, optimizer, train_loader, num_epochs)


# Evaluate the Model and Detect Anomalies

In [ ]:
def compute_anomaly_scores(model, data_loader):
    model.eval()
    anomaly_scores = []
    true_labels = []
    predictions = []

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.unsqueeze(1).to(device)
            y_batch = y_batch.to(device)

            x_rec, class_logits = model(X_batch)
            y_probs = F.softmax(class_logits, dim=1)

            # Reconstruction error
            rec_error = torch.mean((X_batch - x_rec) ** 2, dim=[1, 2])

            # Classification probability for anomaly class
            anomaly_prob = y_probs[:, 1]

            # Combine scores
            anomaly_score = rec_error * anomaly_prob

            anomaly_scores.extend(anomaly_score.cpu().numpy())
            true_labels.extend(y_batch.cpu().numpy())
            predictions.extend(torch.argmax(class_logits, dim=1).cpu().numpy())

    return np.array(anomaly_scores), np.array(true_labels), np.array(predictions)

# Compute anomaly scores
train_anomaly_scores, train_true_labels, train_predictions = compute_anomaly_scores(model, train_loader)


test_anomaly_scores, test_true_labels, test_predictions = compute_anomaly_scores(model, test_loader)
#print(test_anomaly_scores)
info = precision_recall_fscore_support(test_true_labels, test_predictions, pos_label=1)
print("Precision = {}%, recall = {}% and F1-score = {}%".format(np.round(info[0][1]*100, 2),
                                                                np.round(info[1][1]*100, 2),
                                                                np.round(info[2][1]*100, 2)))



In [ ]:
# Define the scale factor
scale = 2  # You can adjust this value
# Calculate the threshold
threshold = np.mean(train_anomaly_scores) + scale * np.std(train_anomaly_scores)
print(f"Anomaly Detection Threshold: {threshold}")
# Classify test data based on threshold
threshold_predictions = (test_anomaly_scores > threshold).astype(int)

# Evaluate threshold-based predictions
print("Threshold-based Classification Report:")
print(classification_report(test_true_labels, threshold_predictions, target_names=['Normal', 'Anomaly']))

# Confusion matrix
cm_threshold = confusion_matrix(test_true_labels, threshold_predictions)
print("Threshold-based Confusion Matrix:")
print(cm_threshold)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(train_anomaly_scores[train_true_labels == 0], bins=50, alpha=0.6, label='Normal')
plt.hist(train_anomaly_scores[train_true_labels == 1], bins=50, alpha=0.6, label='Anomaly')
plt.title('Anomaly Score Distribution')
plt.xlabel('Anomaly Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()


In [ ]:
def plot_anomaly_scores_scatter(anomaly_scores, true_labels, threshold):
    plt.figure(figsize=(10, 6))
    plt.scatter(range(len(anomaly_scores)), anomaly_scores, c=true_labels, cmap='coolwarm', label='Data Point')
    plt.axhline(threshold, color='red', linestyle='--', label='Threshold')
    plt.title('Anomaly Scores Scatter Plot')
    plt.xlabel('Sample Index')
    plt.ylabel('Anomaly Score')
    plt.legend()
    plt.show()

# Call the function
plot_anomaly_scores_scatter(test_anomaly_scores, test_true_labels, threshold)


In [ ]:


# Set plot style
sns.set(style='whitegrid')
def plot_anomaly_score_histogram(anomaly_scores, true_labels, threshold):
    plt.figure(figsize=(10, 6))
    sns.histplot(anomaly_scores[true_labels == 0], bins=50, color='green', label='Normal', kde=True, stat='density')
    sns.histplot(anomaly_scores[true_labels == 1], bins=50, color='red', label='Anomaly', kde=True, stat='density')
    plt.axvline(threshold, color='blue', linestyle='--', label='Threshold')
    plt.title('Histogram of Anomaly Scores')
    plt.xlabel('Anomaly Score')
    plt.ylabel('Density')
    plt.legend()
    plt.show()

# Call the function
plot_anomaly_score_histogram(test_anomaly_scores, test_true_labels, threshold)


In [ ]:


# Compute AUC-ROC
auc_score = roc_auc_score(test_true_labels, test_anomaly_scores)
print(f"AUC-ROC Score: {auc_score:.4f}")

# Compute Precision-Recall Curve
precision, recall, thresholds = precision_recall_curve(test_true_labels, test_anomaly_scores)
ap_score = average_precision_score(test_true_labels, test_anomaly_scores)
print(f"Average Precision Score: {ap_score:.4f}")


In [ ]:
fpr, tpr, thresholds = roc_curve(test_true_labels, test_anomaly_scores)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.4f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()


In [ ]:


# Classification report
print("Classification Report:")
print(classification_report(test_true_labels, test_predictions, target_names=['Normal', 'Anomaly']))

# Confusion matrix
cm = confusion_matrix(test_true_labels, test_predictions)
print("Confusion Matrix:")
print(cm)

# Compute AUC-ROC
auc_score = roc_auc_score(test_true_labels, test_anomaly_scores)
print(f"AUC-ROC Score: {auc_score:.4f}")


In [ ]:


plt.figure(figsize=(10, 6))
plt.hist(test_anomaly_scores[test_true_labels == 0], bins=50, alpha=0.6, label='Normal')
plt.hist(test_anomaly_scores[test_true_labels == 1], bins=50, alpha=0.6, label='Anomaly')
plt.axvline(threshold, color='red', linestyle='--', label='Threshold')
plt.title('Anomaly Score Distribution with Threshold')
plt.xlabel('Anomaly Score')
plt.ylabel('Frequency')
plt.legend()
plt.show()
